# Classification Surrogate Tests

We are interested in testing whether or not a surrogate model can correctly identify unknown constraints based on categorical criteria with classification surrogates. Essentially, we want to account for scenarios where specialists can look at a set of experiments and label outcomes as 'acceptable', 'unacceptable', 'ideal', etc. 

This involves new models that produce `CategoricalOutput`'s rather than continuous outputs. Mathematically, if $g_{\theta}:\mathbb{R}^d\to[0,1]^c$ represents the function governed by learnable parameters $\theta$ which outputs a probability vector over $c$ potential classes (i.e. for input $x\in\mathbb{R}^d$, $g_{\theta}(x)^\top\mathbf{1}=1$ where $\mathbf{1}$ is the vector of all 1's) and we have acceptibility criteria for the corresponding classes given by $a\in\{0,1\}^c$, we can compute the scalar output $g_{\theta}(x)^\top a\in[0,1]$ which represents the expected value of acceptance as an objective value to be passed in as a constrained function.

In this script, we look at the [Rosenbrock function constrained to a disk](https://en.wikipedia.org/wiki/Test_functions_for_optimization#cite_note-12) which attains a global minima at $(x_0^*,x_1^*)=(1.0, 1.0)$. To facilitate testing the functionality offered by BoFire, we label all points inside of the circle $x_0^2+x_1^2\le2$ as 'acceptable' and futher label anything inside of the interesction of this circle and the circle $(x_0-1)^2+(x_1-1)^2\le1.0$ as 'ideal'; points lying outside of these two locations are labeled as "unacceptable."

In [ ]:
# Import packages
import numpy as np
import pandas as pd

import bofire.strategies.api as strategies
from bofire.data_models.api import Domain, Inputs, Outputs
from bofire.data_models.features.api import (
    CategoricalInput,
    CategoricalOutput,
    ContinuousInput,
    ContinuousOutput,
)
from bofire.data_models.objectives.api import (
    ConstrainedCategoricalObjective,
    MinimizeObjective,
    MinimizeSigmoidObjective,
)

## Manual setup of the optimization domain

The following cells show how to manually setup the optimization problem in BoFire for didactic purposes.

In [ ]:
# Write helper functions which give the objective and the constraints
def rosenbrock(x: pd.Series) -> pd.Series:
    assert "x_0" in x.columns
    assert "x_1" in x.columns
    return (1 - x["x_0"]) ** 2 + 100 * (x["x_1"] - x["x_0"] ** 2) ** 2


def constraints(x: pd.Series) -> pd.Series:
    assert "x_0" in x.columns
    assert "x_1" in x.columns
    feasiblity_vector = []
    for _, row in x.iterrows():
        if (row["x_0"] ** 2 + row["x_1"] ** 2 <= 2.0) and (
            (row["x_0"] - 1.0) ** 2 + (row["x_1"] - 1.0) ** 2 <= 1.0
        ):
            feasiblity_vector.append("ideal")
        elif row["x_0"] ** 2 + row["x_1"] ** 2 <= 2.0:
            feasiblity_vector.append("acceptable")
        else:
            feasiblity_vector.append("unacceptable")
    return feasiblity_vector

In [ ]:
# Set-up the inputs and outputs, use categorical domain just as an example
input_features = Inputs(
    features=[ContinuousInput(key=f"x_{i}", bounds=(-1.75, 1.75)) for i in range(2)]
    + [CategoricalInput(key="x_3", categories=["0", "1"], allowed=[True, True])]
)

# here the minimize objective is used, if you want to maximize you have to use the maximize objective.
output_features = Outputs(
    features=[
        ContinuousOutput(key=f"f_{0}", objective=MinimizeObjective(w=1.0)),
        CategoricalOutput(
            key=f"f_{1}",
            categories=["unacceptable", "acceptable", "ideal"],
            objective=ConstrainedCategoricalObjective(
                categories=["unacceptable", "acceptable", "ideal"],
                desirability=[False, True, True],
            ),
        ),  # This function will be associated with learning the categories
        ContinuousOutput(
            key=f"f_{2}",
            objective=MinimizeSigmoidObjective(w=1.0, tp=0.0, steepness=0.5),
        ),
    ]
)

# Create domain
domain1 = Domain(inputs=input_features, outputs=output_features)

# Sample random points
sample_df = domain1.inputs.sample(100)

# Write a function which outputs one continuous variable and another discrete based on some logic
sample_df["f_0"] = rosenbrock(x=sample_df)
sample_df["f_1"] = constraints(x=sample_df)
sample_df["f_2"] = sample_df["x_3"].astype(float) + 1e-2 * np.random.uniform(
    size=(len(sample_df),)
)
sample_df.head(5)

,x_0,x_1,x_3,f_0,f_1,f_2
0,-0.073213,-0.173816,0,4.362194,acceptable,0.007990
1,-0.558842,0.940771,0,41.927069,acceptable,0.005296
2,-1.663016,-0.422555,0,1023.539326,unacceptable,0.005456
3,-1.286946,1.150658,0,30.790510,unacceptable,0.007460
4,0.756959,0.342372,0,5.377383,ideal,0.003363


In [ ]:
# Plot the sample df
import math

import plotly.express as px


fig = px.scatter(
    sample_df,
    x="x_0",
    y="x_1",
    color="f_1",
    width=550,
    height=525,
    title="Samples with labels",
)
fig.add_shape(
    type="circle",
    xref="x",
    yref="y",
    opacity=0.1,
    fillcolor="red",
    x0=-math.sqrt(2),
    y0=-math.sqrt(2),
    x1=math.sqrt(2),
    y1=math.sqrt(2),
    line_color="red",
)
fig.add_shape(
    type="circle",
    xref="x",
    yref="y",
    opacity=0.2,
    fillcolor="LightSeaGreen",
    x0=0,
    y0=0,
    x1=2,
    y1=2,
    line_color="LightSeaGreen",
)
fig.show()

## Evaluate the classification model performance (outside of the optimization procedure)

In [ ]:
# Import packages
import bofire.surrogates.api as surrogates
from bofire.data_models.surrogates.api import ClassificationMLPEnsemble
from bofire.surrogates.diagnostics import ClassificationMetricsEnum


# Instantiate the surrogate model
model = ClassificationMLPEnsemble(
    inputs=domain1.inputs,
    outputs=Outputs(features=[domain1.outputs.get_by_key("f_1")]),
    lr=0.03,
    n_epochs=100,
    hidden_layer_sizes=(
        4,
        2,
    ),
    weight_decay=0.0,
    batch_size=10,
    activation="tanh",
)
surrogate = surrogates.map(model)

# Fit the model to the classification data
cv_df = sample_df.drop(["f_0", "f_2"], axis=1)
cv_df["valid_f_1"] = 1
cv = surrogate.cross_validate(cv_df, folds=3)

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\bo

In [ ]:
# Print results
cv[0].get_metrics(
    metrics=ClassificationMetricsEnum, combine_folds=True
)  # print training set performance

,ACCURACY,F1
0,0.725,0.725


In [ ]:
cv[1].get_metrics(
    metrics=ClassificationMetricsEnum, combine_folds=True
)  # print test set performance

,ACCURACY,F1
0,0.56,0.56


## Setup strategy and ask for candidates



In [ ]:
from bofire.data_models.acquisition_functions.api import qLogEI
from bofire.data_models.domain.api import Outputs
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.surrogates.api import (
    BotorchSurrogates,
    ClassificationMLPEnsemble,
    MixedSingleTaskGPSurrogate,
)


strategy_data = SoboStrategy(
    domain=domain1,
    acquisition_function=qLogEI(),
    surrogate_specs=BotorchSurrogates(
        surrogates=[
            ClassificationMLPEnsemble(
                inputs=domain1.inputs,
                outputs=Outputs(features=[domain1.outputs.get_by_key("f_1")]),
                lr=0.03,
                n_epochs=100,
                hidden_layer_sizes=(
                    4,
                    2,
                ),
                weight_decay=0.0,
                batch_size=10,
                activation="tanh",
            ),
            MixedSingleTaskGPSurrogate(
                inputs=domain1.inputs,
                outputs=Outputs(features=[domain1.outputs.get_by_key("f_2")]),
            ),
        ]
    ),
)

strategy = strategies.map(strategy_data)

strategy.tell(sample_df)

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\optim\fit.py:102: OptimizationWarning:

`scipy_minimize` terminated with status 3, displaying original message from `scipy.optimize.minimize`: ABNORMAL_TERMINATION_IN_LNSRCH



In [ ]:
candidates = strategy.ask(10)
candidates

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.



,x_0,x_1,x_3,f_1_pred,f_1_sd,f_1_unacceptable_prob,f_1_acceptable_prob,f_1_ideal_prob,f_0_pred,f_2_pred,f_1_unacceptable_sd,f_1_acceptable_sd,f_1_ideal_sd,f_0_sd,f_2_sd,f_0_des,f_2_des,f_1_des
0,0.445121,0.203488,1,acceptable,0.0,-0.193104,1.006308,0.104767,0.144256,0.750977,4.968438,0.003073,0.203727,0.219485,0.421818,-0.144256,0.407216,1.111075
1,-1.309524,1.750000,0,unacceptable,0.0,0.730642,0.006611,0.368515,0.630446,0.001039,16.560211,0.003667,0.348812,0.347461,0.001424,-0.630446,0.499870,0.375126
2,-1.258058,1.750000,1,acceptable,0.0,-10.671111,1.006274,0.339735,0.471522,0.188743,19.401399,0.003533,0.410371,0.431353,0.417536,-0.471522,0.476425,1.346009
3,1.292107,1.750000,1,acceptable,0.0,-5.226207,1.005614,0.263137,0.002680,0.734182,6.172339,0.003671,0.427584,0.003406,0.425910,-0.002680,0.409244,1.268751
4,0.498312,0.254918,0,ideal,0.0,0.202863,0.005522,0.363525,0.237623,0.398852,5.107777,0.003061,0.343516,0.236302,0.546131,-0.237623,0.450308,0.369047
5,-1.430995,1.750000,0,unacceptable,0.0,15.038112,0.007034,0.367135,0.631857,0.001008,22.077586,0.003766,0.347731,0.346423,0.001392,-0.631857,0.499874,0.374169
6,-1.501348,1.627440,0,unacceptable,0.0,49.375552,0.007131,0.364155,0.634857,0.000988,21.504947,0.003674,0.345456,0.344176,0.001372,-0.634857,0.499877,0.371287
7,-1.468474,1.695817,0,unacceptable,0.0,29.651239,0.007100,0.365440,0.633564,0.000996,22.143859,0.003728,0.346425,0.345133,0.001380,-0.633564,0.499875,0.372539
8,-1.518190,1.517966,0,unacceptable,0.0,72.020318,0.007057,0.363156,0.635863,0.000981,18.759220,0.003571,0.344707,0.343437,0.001366,-0.635863,0.499877,0.370213
9,-1.459283,1.442226,0,unacceptable,0.0,54.177939,0.006770,0.362903,0.636116,0.000981,13.822711,0.003457,0.344511,0.343241,0.001366,-0.636116,0.499877,0.369673


## Check classification of proposed candidates

Use the logic from above to verify the classification values

In [ ]:
# Append to the candidates
candidates["f_1_true"] = constraints(x=candidates)

In [ ]:
# Print results
candidates[["x_0", "x_1", "f_1_pred", "f_1_true"]]

,x_0,x_1,f_1_pred,f_1_true
0,0.445121,0.203488,acceptable,ideal
1,-1.309524,1.750000,unacceptable,unacceptable
2,-1.258058,1.750000,acceptable,unacceptable
3,1.292107,1.750000,acceptable,unacceptable
4,0.498312,0.254918,ideal,ideal
5,-1.430995,1.750000,unacceptable,unacceptable
6,-1.501348,1.627440,unacceptable,unacceptable
7,-1.468474,1.695817,unacceptable,unacceptable
8,-1.518190,1.517966,unacceptable,unacceptable
9,-1.459283,1.442226,unacceptable,unacceptable
